In [1]:
## La version de python utilisée est 3.12.7

!pip install -r requirements.txt -q

In [ ]:
import pandas as pd
import requests as rq
import lxml as lxml
from bs4 import BeautifulSoup
import io as io
import math
import gzip
import shutil
import os
import geopandas as gpd
import matplotlib.pyplot as plt
import folium
import json
from pandasgui import show
import numpy as np
from io import BytesIO
from folium.plugins import HeatMap
import nbconvert
import numpy as np
import statsmodels.formula.api as smf
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor

from script import process_data
from script import geolocaliser
from script import modeling
from geopy.geocoders import Nominatim
from geopy.exc import GeocoderTimedOut

In [ ]:
df = pd.read_csv("data/transactions_with_zone_inondable.csv",encoding="utf-8",sep=";")

C:\Users\flori\AppData\Local\Temp\ipykernel_4516\648495320.py:1: DtypeWarning:

Columns (6,8) have mixed types. Specify dtype option on import or set low_memory=False.



In [ ]:
show(df.head())

PandasGUI INFO — pandasgui.gui — Opening PandasGUI
C:\Users\flori\AppData\Local\Temp\ipykernel_4516\3342683114.py:1: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`

C:\Users\flori\AppData\Local\Temp\ipykernel_4516\3342683114.py:1: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`

C:\Users\flori\AppData\Local\Temp\ipykernel_4516\3342683114.py:1: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`

C:\Users\flori\AppData\Local\Temp\ipykernel_4516\

In [ ]:
# Calcul de la distance pour chaque ligne du DataFrame
df['distance_centre_ville'] = df.apply(
    lambda row: modeling.distance_haversine(
        row['latitude'], row['longitude'],
        row['latitude_mairie'], row['longitude_mairie']
    ),
    axis=1
)

In [ ]:
import ast

# Conversion des chaînes en listes de tuples
df["beach_coordinates"] = df["beach_coordinates"].apply(
    lambda x: ast.literal_eval(x) if isinstance(x, str) else x
)

df["station"] = df["station"].apply(
    lambda x: ast.literal_eval(x) if isinstance(x, str) else x
)

In [ ]:
def nettoyer_coordinates(coords):
    if isinstance(coords, list):  # Si c'est déjà une liste, la garder
        return coords
    if isinstance(coords, float) or coords is None:  # Si c'est un NaN ou autre, retourner None
        return None
    return coords  # Par défaut, retourner la valeur initiale (si elle est correcte)

df["beach_coordinates"] = df["beach_coordinates"].apply(nettoyer_coordinates)
df["station"] = df["station"].apply(nettoyer_coordinates)

In [ ]:
# Calcul de la distance minimale à la plage ou au port en fonction de la disponibilité des données
df["distance_min_beach"] = df.apply(
    lambda row: modeling.distance_minimale(row["latitude"], row["longitude"], row["beach_coordinates"])
    if row["beach_coordinates"] else None,  # Si aucune coordonnée plage disponible, retourner None
    axis=1
)

# Si distance_min_beach est NaN, calculer la distance au port
df["distance_min_beach"] = df.apply(
    lambda row: modeling.distance_haversine(
        row['latitude'], row['longitude'],
        row['latitude_port'], row['longitude_port']
    ) if pd.isna(row["distance_min_beach"]) else row["distance_min_beach"],
    axis=1
)


In [ ]:
df["distance_min_station"] = df.apply(
    lambda row: modeling.distance_minimale(row["latitude"], row["longitude"], row["station"])
    if row["station"] else None,  # Si aucune coordonnée disponible, retourner None
    axis=1
)

In [ ]:
def convertir_en_numerique(df, colonnes):
    """
    Convertit les colonnes spécifiées du DataFrame en type numérique.
    
    Args:
        df (pd.DataFrame): Le DataFrame à modifier.
        colonnes (list): Liste des colonnes à convertir en numériques.
        
    Returns:
        pd.DataFrame: Le DataFrame avec les colonnes converties.
    """
    for col in colonnes:
        df[col] = pd.to_numeric(df[col], errors='coerce')
    return df

In [ ]:
# Affichage des modalités de 'code_scenario'
modalites_code_scenario = df['code_scenario'].value_counts()
print(modalites_code_scenario)

code_scenario
02Moy    2839
04Fai    2423
01For    1248
03Mcc    1052
Name: count, dtype: int64


In [ ]:
# Définir les bornes des tranches et les labels
bins = [0, 10000, 50000, np.inf]  # Définir les bornes des tranches
labels = ['0-10000','10000-50000', 'plus_50000']  # Labels des tranches

# Créer la variable catégorielle 'pop'
df['pop_cut'] = pd.cut(df['Population'], bins=bins, labels=labels, right=False)

# Créer des variables dummies pour les catégories de population
df_dummies_population = pd.get_dummies(df['pop_cut'], prefix='population_',drop_first=True)
df_dummies_population = df_dummies_population.astype(int)

# Ajouter les dummies au DataFrame
df = pd.concat([df, df_dummies_population], axis=1)

In [ ]:
# Création des dummies pour la colonne 'code_scenario'
df_dummies_scenario = pd.get_dummies(df['code_scenario'], prefix='scenario')
# df_dummies_scenario = df_dummies_scenario.drop(columns=['scenario_04Fai'])
df_dummies_scenario = df_dummies_scenario.astype(int)

# Ajouter les dummies au DataFrame original
df = pd.concat([df, df_dummies_scenario], axis=1)


In [ ]:
# Création des dummies pour la colonne 'departmentName'
df_dummies_communes = pd.get_dummies(df['nom_commune'], prefix='commune')
# # Si vous souhaitez supprimer une modalit/é particulière, vous pouvez utiliser .drop() :
df_dummies_communes = df_dummies_communes.drop(columns=['commune_Nice'])

# # Convertir les dummies en entiers (0/1)
df_dummies_communes = df_dummies_communes.astype(int)

# # Ajouter les dummies pour 'departmentName' au DataFrame original
df = pd.concat([df, df_dummies_communes], axis=1)

In [ ]:
# Création des dummies pour la colonne 'departmentName'
df_dummies_department = pd.get_dummies(df['departmentName'], prefix='department')
# # Si vous souhaitez supprimer une modalit/é particulière, vous pouvez utiliser .drop() :
df_dummies_department = df_dummies_department.drop(columns=['department_Alpes-Maritimes'])

# # Convertir les dummies en entiers (0/1)
df_dummies_department = df_dummies_department.astype(int)

# # Ajouter les dummies pour 'departmentName' au DataFrame original
df = pd.concat([df, df_dummies_department], axis=1).drop(columns='departmentName')

In [ ]:
show(df_mai.head(100))

PandasGUI INFO — pandasgui.gui — Opening PandasGUI
C:\Users\flori\AppData\Local\Temp\ipykernel_4516\4000138087.py:1: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`

C:\Users\flori\AppData\Local\Temp\ipykernel_4516\4000138087.py:1: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`

C:\Users\flori\AppData\Local\Temp\ipykernel_4516\4000138087.py:1: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`

C:\Users\flori\AppData\Local\Temp\ipykernel_4516\

In [ ]:
# Transformation en log de certaines colonnes
variables_continues =    ["surface_reelle_bati",
                          "prix_m2",
                          "distance_min_beach",
                          "distance_min_station",
                          "distance_centre_ville"
                          ]

df = modeling.transformer_log(df, variables_continues)

df_app = df[df["type_local"]=="Appartement"]
df_mai = df[df["type_local"]=="Maison"]

# Colonnes explicatives, simplifiées à partir de variables_continues
colonnes_explicatives = [
    "log_surface_reelle_bati",
    "log_distance_min_beach",
    # "nombre_pieces_principales",
    "log_distance_min_station",
    "log_distance_centre_ville",
    "zone_inondable",
    # "scenario_01For"
] + [
    col for col in df.columns if col.startswith('population')
] + [
#     col for col in df.columns if col.startswith('scenario')
] + [
#     col for col in df.columns if col.startswith('department')
# ] + [
    col for col in df.columns if col.startswith('commune_')
]

colonnes_explicatives2 = [
    col for col in colonnes_explicatives if col != "zone_inondable"
] + [
    col for col in df.columns if col.startswith("scenario")
    
]

colonne_dependante = "log_prix_m2"

# Création du modèle avec dummies pour les départements
model_app = modeling.construire_modele_regression(
    df_app,
    colonnes_explicatives=colonnes_explicatives,
    colonne_dependante=colonne_dependante
)

model_app2 = modeling.construire_modele_regression(
    df_app,
    colonnes_explicatives=colonnes_explicatives2,
    colonne_dependante=colonne_dependante
)

model_mai = modeling.construire_modele_regression(
    df_mai,
    colonnes_explicatives=colonnes_explicatives,
    colonne_dependante=colonne_dependante
)

model_mai2 = modeling.construire_modele_regression(
    df_mai,
    colonnes_explicatives=colonnes_explicatives2,
    colonne_dependante=colonne_dependante
)
# Ajustement avec covariance robuste de type HC3
# model_robust = model.get_robustcov_results(cov_type='HC3')
# vif_data["VIF"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]

# Résumé des résultats
# print(model_robust.summary())
# Résultats du modèle
# print(model_app.summary())
# print(model_mai.summary())
# print(vif_data)


In [ ]:
# Extraire les résultats des 4 modèles
resultats_app = modeling.extraire_resultats_modele(model_app)
resultats_app2 = modeling.extraire_resultats_modele(model_app2)
resultats_mai = modeling.extraire_resultats_modele(model_mai)
resultats_mai2 = modeling.extraire_resultats_modele(model_mai2)

# Fusionner les résultats des modèles sur le nom des variables
tableau_resultats = (
    resultats_app[["variable"]]
    .merge(resultats_app, on="variable", how="outer", suffixes=("", "_app"))
    .merge(resultats_app2, on="variable", how="outer", suffixes=("_app", "_app2"))
    .merge(resultats_mai, on="variable", how="outer", suffixes=("_app2", "_mai"))
    .merge(resultats_mai2, on="variable", how="outer", suffixes=("_mai", "_mai2"))
)

# Renommer les colonnes pour clarté
tableau_resultats = tableau_resultats.rename(columns={
    "coefficient": "Coeff_app", "significativite": "Signif_app",
    "coefficient_app2": "Coeff_app2", "significativite_app2": "Signif_app2",
    "coefficient_mai": "Coeff_mai", "significativite_mai": "Signif_mai",
    "coefficient_mai2": "Coeff_mai2", "significativite_mai2": "Signif_mai2",
})

# Réorganiser les colonnes pour affichage
tableau_final = tableau_resultats[[
    "variable", 
    "Coeff_app", "Signif_app", 
    "Coeff_app2", "Signif_app2", 
    "Coeff_mai", "Signif_mai", 
    "Coeff_mai2", "Signif_mai2"
]]

# Affichage du tableau final
print(tableau_final)

AttributeError: module 'script.modeling' has no attribute 'extraire_resultats_modele'